In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spam_detection").getOrCreate()

In [3]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection", inferSchema=True, sep='\t')

In [6]:
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn("length", length(data["text"]))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [11]:
data.groupBy("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [14]:
stop_remove = StopWordsRemover(inputCol="token_text", outputCol="stop_token")

In [15]:
count_vec = CountVectorizer(inputCol="stop_token", outputCol="c_vec")

In [16]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [17]:
ham_spam_to_numeric = StringIndexer(inputCol="class", outputCol="label")

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
clean_up = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

In [22]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer,
                                stop_remove, count_vec, idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

In [26]:
clean_data = cleaner.transform(data)

In [27]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|          stop_token|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [28]:
clean_data = clean_data.select("label", "features")

In [29]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [30]:
train, test = clean_data.randomSplit([0.7, 0.3])

In [31]:
spam_detector = nb.fit(train)

In [32]:
test_results = spam_detector.transform(test)

In [33]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-792.04305474472...|[1.0,5.0717073004...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-613.22667108589...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-573.99989249585...|[1.0,5.1253397207...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-847.97053985551...|[1.0,2.0204113989...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-999.89889936044...|[1.0,1.4300536880...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-812.38563810840...|[1.0,9.4072754719...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1378.4922253117...|[1.0,1.1209791261...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-690.04514328701...|[1.0,2.5890271437...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-825.78712222559...|[1.0,2.4426867537...|       0.0|
|  0.0|(13424,[0

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
acc_eval = MulticlassClassificationEvaluator()

In [36]:
acc = acc_eval.evaluate(test_results)

In [37]:
acc

0.9197948455788038